# Profitable App Profiles for the App Store and Google Play Markets

## Phase 1: Business Understanding
_The goal of this project is to determine the kind of apps an app development company should focus their efforts on considering various factors such as market share, competition, app genre, and user engagement._

---

## Phase 2: Data Mining
_Finding necessary data to fit the purpose of analysis_

### Data sets used
_Further information/documentation are available in the source links._
* [Android](https://www.kaggle.com/lava18/google-play-store-apps/home) contains data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018.
* [iOS](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) contains data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017.

In [1]:
from csv import reader

### Android data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_raw = list(read_file)
android_header = android_raw[0]
android_data = android_raw[1:]

### iOS data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios_raw = list(read_file)
ios_header = ios_raw[0]
ios_data = ios_raw[1:]

In [2]:
def convert_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
### Android preview ###
print(android_header)
print('\n')
convert_data(android_data, 0, 1, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In [4]:
### iOS preview ###
print(ios_header)
print('\n')
convert_data(ios_data, 0, 1, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7197
Number of columns: 16


### Relevant Info Between These Data Sets

| Android Index | Android Column | iOS Index | iOS Column |
| --- | --- | --- | --- |
| 0 | App | 1 | track_name |
| 1 | Category | 11 | prime_genre |
| 2 | Rating | 7 | user_rating |
| 3 | Reviews | 5 | rating_count_tot |
| 7 | Price | 4 | price |
| 8 | Content Rating | 10 | cont_rating |

---

## Phase 3: Data Cleaning
_Making sure data across the boards are complete and consistent by:_
* deleting wrong data
* removing duplicates
* removing non-english apps
* isolating free apps

### Deleting Wrong Data

In [5]:
print(android_header)
print(android_data[10472]) # missing CATEGORY value
# del android_data[10472]  <--- converted to comment after running once
print(android_data[10472]) # after deletion, new len should be 10840

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


### Removing Duplicates

In [6]:
### Checking for duplicates in android data ###

duplicate_android = []
unique_android = []

for app in android_data:
    name = app[0]
    if name in unique_android:
        duplicate_android.append(name)
    else:
        unique_android.append(name)
        
print('Duplicate android apps: ', len(duplicate_android))
print('\n')
print('Including: ', duplicate_android[:10])

### Keeping only duplicate with the most reviews
n_reviews_max = {} # starts with an empty dictionary
for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in n_reviews_max and n_reviews_max[name] < n_reviews:
        n_reviews_max[name] = n_reviews # if the current n_reviews_max is less, we change the value
    elif name not in n_reviews_max:
        n_reviews_max[name] = n_reviews # otherwise, we add a new key-value pair

Duplicate android apps:  1181


Including:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


We expect there to be 1181 duplicates. So after deleting entries, the **new len of android_data should be 9659** (previous len minus duplicates).

In [7]:
len(n_reviews_max)

9659

In [8]:
### Isolating android data to a new data set

android_clean = [] # stores new cleaned data set
android_already_added = [] # only store app names

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews_max[name] == n_reviews) and (name not in android_already_added):
        android_clean.append(app)
        android_already_added.append(app[0])
        
print('len of android_data:', len(android_data))
print('len of android_clean: ', len(android_clean))
print('number of entries removed: ', len(android_data) - len(android_clean))


len of android_data: 10840
len of android_clean:  9659
number of entries removed:  1181


In [9]:
### Checking for duplicates in ios data ###

duplicate_ios = []
unique_ios = []

for app in ios_data:
    name = app[1]
    if name in unique_ios:
        duplicate_ios.append(name)
    else:
        unique_ios.append(name)
        
print('Duplicate ios apps: ', len(duplicate_ios))
print('\n')
print('Including: ', duplicate_ios[:10])

### Keeping only duplicate with the most reviews
n_reviews_max_ios = {} # starts with an empty dictionary
for app in ios_data:
    name = app[1]
    n_reviews = float(app[5])
    if name in n_reviews_max_ios and n_reviews_max_ios[name] < n_reviews:
        n_reviews_max_ios[name] = n_reviews # if the current n_reviews_max is less, we change the value
    elif name not in n_reviews_max_ios:
        n_reviews_max_ios[name] = n_reviews # otherwise, we add a new key-value pair

Duplicate ios apps:  2


Including:  ['Mannequin Challenge', 'VR Roller Coaster']


We expect there to be 2 duplicates. So after deleting entries, the **new len of ios_data should be 7195** (previous len minus duplicates).

In [10]:
len(n_reviews_max_ios)

7195

In [11]:
### Isolating ios data to a new data set

ios_clean = [] # stores new cleaned data set
ios_already_added = [] # only store app names

for app in ios_data:
    name = app[1]
    n_reviews = float(app[5])
    
    if (n_reviews_max_ios[name] == n_reviews) and (name not in ios_already_added):
        ios_clean.append(app)
        ios_already_added.append(app[1])
        
print('len of ios_data:', len(ios_data))
print('len of ios_clean: ', len(ios_clean))
print('number of entries removed: ', len(ios_data) - len(ios_clean))

len of ios_data: 7197
len of ios_clean:  7195
number of entries removed:  2


Now, we have a separate and clean data sets called `android_clean` and `ios_clean`. We did this by following these steps:
1. Checking for duplicates
2. Getting duplicates with the most reviews
3. Isolating the apps we need in new data sets:
    * unique apps
    * duplicates with most reviews
4. Validating that we have to correct and expected len for both data sets

### Removing non-English apps
We develop apps directed for English-speaking audience so there's no need to include in both data sets apps that are not targeted to the same audience.

Characters that are not typically used in English text can be isolated by using the `ord(character)` function. Originally based on the English alphabet, ASCII encodes __128 specified characters (slice :127)__ into seven-bit integers as shown by the ASCII chart below:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/USASCII_code_chart.png/361px-USASCII_code_chart.png" alt="Alt text that describes the graphic" title="Title text" />

We will create a function called `is_this_english(string)` that will take a string and returns the bool `True` or `False` based on whether there are characters that suggest the app is not directed for English-speaking audience.

In [12]:
### Creating the is_this_english function ###
# this will return False only if there are more than 3 characters
# within the string classified as non_ascii

def is_english(string):
    non_ascii = 0
    
    for char in string:
        if ord(char) > 127:
            non_ascii += 1
            
    if non_ascii > 3:
        return False
    else:
        return True

is_english('爱奇艺PPS -《欢乐颂2》电视剧热播') # test for is_english function

False

In [13]:
android_english = []
for app in android_clean:
    if is_english(app[0]):
        android_english.append(app)

print('len of android_clean:', len(android_clean))
print('len of android_english: ', len(android_english))
print('number of entries removed: ', len(android_clean) - len(android_english))
        
ios_english = []
for app in ios_clean:
    if is_english(app[1]):
        ios_english.append(app)

print('\n')
print('len of ios_clean:', len(ios_clean))
print('len of ios_english: ', len(ios_english))
print('number of entries removed: ', len(ios_clean) - len(ios_english))

len of android_clean: 9659
len of android_english:  9614
number of entries removed:  45


len of ios_clean: 7195
len of ios_english:  6181
number of entries removed:  1014


### Isolating free apps
We will to this by looping through the price columns in both data sets: `android_english[app][7]` and `ios_english[app][4]`.

In [14]:
### Isolating free apps on android ###

android_free = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_free.append(app)
        
print('len of android_english: ', len(android_english))
print('len of android_free:', len(android_free))
print('number of entries removed: ', len(android_english) - len(android_free))

len of android_english:  9614
len of android_free: 8864
number of entries removed:  750


In [15]:
### Isolating free apps on ios ###

ios_free = []

for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_free.append(app)
        
print('len of ios_english: ', len(ios_english))
print('len of ios_free:', len(ios_free))
print('number of entries removed: ', len(ios_english) - len(ios_free))

len of ios_english:  6181
len of ios_free: 3220
number of entries removed:  2961


### Clean up funnel

| Derived list | Android | iOS |
| --- | --- |
| _data | 10841 | 7197 |
| (remove wrong data) | 10840 | 7197 |
| _clean | 9659 | 7195 |
| _english | 9614 | 6191 |
| _free | 8864 | 3220 |

---

## Phase 4: Data Exploration
_Form hypothesis to answer what kind of apps we should be building_

### Validation strategy
To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

### Ways to explore data
1. Most common overall genres by number of apps - _what kind of apps do people look for?_
2. Most engaging genres by number of reviews - _what do users use and care about?_
3. Most common apps per recommended genre - _what specific apps can we look into?_

In [16]:
### Most common overall genres by number of apps ###
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for app in dataset:
        total += 1
        value = app[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for value in table:
        percentage = (table[value] / total) * 100
        table_percentages[value] = percentage 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [17]:
android_genre = display_table(android_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [18]:
ios_genre = display_table(ios_free, 11)

Games : 58.13664596273293
Entertainment : 7.888198757763975
Photo & Video : 4.968944099378882
Education : 3.6645962732919255
Social Networking : 3.291925465838509
Shopping : 2.608695652173913
Utilities : 2.515527950310559
Sports : 2.142857142857143
Music : 2.049689440993789
Health & Fitness : 2.018633540372671
Productivity : 1.7391304347826086
Lifestyle : 1.5838509316770186
News : 1.3354037267080745
Travel : 1.2422360248447204
Finance : 1.1180124223602486
Weather : 0.8695652173913043
Food & Drink : 0.8074534161490683
Reference : 0.5590062111801243
Business : 0.5279503105590062
Book : 0.43478260869565216
Navigation : 0.18633540372670807
Medical : 0.18633540372670807
Catalogs : 0.12422360248447205


__iOS App Store is dominated by leisure apps__ with GAMES taking up more than half of the pie (58.14%). Combining it with other categories related to leisure such as ENTERTAINMENT, LIFESTYLE, TRAVEL, FOOD & DRINK (61.77%), the remaining 38.23% is shared by more practical categories including EDUCATION, PRODUCTIVITY, and BUSINESS.

__Google Play has a more balanced landscape__ with FAMILY, GAMES, and TOOLS being most common. However, the differences are less pronounced than they are in iOS App Store.

In [19]:
### Most engaging genres by number of reviews ###

print('Most reviewed Android Apps')
print('\n')
for genre in freq_table(android_free, 1):
    total = 0
    len_genre = 0
    for app in android_free:
        genre_app = app[1]
        if genre_app == genre:            
            n_ratings = float(app[3])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

print('\n')
print('Most reviewed iOS Apps')
print('\n')
    
for genre in freq_table(ios_free, 11):
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[11]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Most reviewed Android Apps


MEDICAL : 3730.1533546325877
VIDEO_PLAYERS : 425350.08176100627
PERSONALIZATION : 181122.31632653062
PRODUCTIVITY : 160634.5420289855
SHOPPING : 223887.34673366835
FAMILY : 113142.99821002387
BUSINESS : 24239.727272727272
ENTERTAINMENT : 301752.24705882353
FOOD_AND_DRINK : 57478.79090909091
ART_AND_DESIGN : 24699.42105263158
EDUCATION : 56293.09708737864
MAPS_AND_NAVIGATION : 142860.0483870968
LIBRARIES_AND_DEMO : 10925.807228915663
WEATHER : 171250.77464788733
FINANCE : 38535.8993902439
PARENTING : 16378.706896551725
EVENTS : 2555.84126984127
PHOTOGRAPHY : 404081.3754789272
TOOLS : 305732.8973333333
NEWS_AND_MAGAZINES : 93088.03225806452
DATING : 21953.272727272728
COMMUNICATION : 995608.4634146341
BEAUTY : 7476.226415094339
HEALTH_AND_FITNESS : 78094.9706959707
SPORTS : 116938.6146179402
COMICS : 42585.61818181818
SOCIAL : 965830.9872881356
GAME : 683523.8445475638
AUTO_AND_VEHICLES : 14140.280487804877
BOOKS_AND_REFERENCE : 87995.06842105264
TRAVEL_AND_L

### Most Rated Android Genre 
_We will focus our development efforts on Android. Higher reward across the board than iOS (downloads, competition, user engagement)._
1. COMMUNICATION : 995608.4634146341
2. SOCIAL : 965830.9872881356
3. GAME : 683523.8445475638 **also among with the most apps**
4. VIDEO_PLAYERS : 425350.081761006
5. PHOTOGRAPHY : 404081.3754789272 **category with fewer apps but more invested users**
6. TOOLS : 305732.8973333333 **moderate number of apps and good engagement**
7. ENTERTAINMENT : 301752.24705882353
8. SHOPPING : 223887.34673366835
9. PERSONALIZATION : 181122.31632653062
10. WEATHER : 171250.77464788733
11. PRODUCTIVITY : 160634.542028985527
12. MAPS_AND_NAVIGATION : 142860.0483870968

### Most Rated iOS Genre 
1. Navigation : 86090.33333333333
2. Reference : 74942.11111111111
3. Social Networking : 71548.34905660378
4. Music : 57326.530303030304 **good competition and engagement**
5. Weather : 52279.892857142855
6. Book : 39758.5
7. Food & Drink : 33333.92307692308
8. Finance : 31467.944444444445
9. Photo & Video : 28441.54375
10. Travel : 28243.8
11. Shopping : 26919.690476190477
12. Health & Fitness : 23298.015384615384

In [20]:
### Most common apps per recommended genre ###

print('Most reviewed Android GAME apps')
for app in android_free:
    genre = app[1]
    n_ratings = float(app[3])
    if genre == 'GAME' and n_ratings > 20000000:
        print(app[0], ':', app[3])
        
print('\n')

print('Most reviewed Android PHOTOGRAPHY apps')
for app in android_free:
    genre = app[1]
    n_ratings = float(app[3])
    if genre == 'PHOTOGRAPHY' and n_ratings > 5000000:
        print(app[0], ':', app[3])

Most reviewed Android GAME apps
Candy Crush Saga : 22430188
Subway Surfers : 27725352
Clash Royale : 23136735
Clash of Clans : 44893888


Most reviewed Android PHOTOGRAPHY apps
B612 - Beauty & Filter Camera : 5282578
Google Photos : 10859051
Retrica : 6120977
PicsArt Photo Studio: Collage Maker & Pic Editor : 7594559
PhotoGrid: Video & Pic Collage Maker, Photo Editor : 7529865


---

## Phase 5: Insights

### Recommended App Profiles to Generate Profit

#### GAMES
* High competition
* Enough people who are interested and engaged (in terms of leaving feedback)
* Ads can be put strategically (to earn extra life, power up, or currency within the game)
    * in-your-face ads - while the next game loads
    * user-activated ads - reward user behavior
* Digital product (vs SHOPPING apps that have engagement but also demands a lot of offline work)

#### PHOTOGRAPHY
* Less competition
* High engagement from users
* Ads can be put strategically
    * limited edition/ bonus stickers/filters
* Digital product (everything is in the app)

<br>

<div class="alert alert-block alert-info">
<b>Note:</b> The results between Android and iOS apps have marked differences in terms of categories. Because Android still has a dominant worldwide market share of 74.85% by April 2019, we decided to focus our development efforts on leading Android categories. 

<br>

Since <b>GAMES</b> and <b>PHOTOGRAPHY</b> are social apps-- meaning users can be very public about the specific apps they use, we can park our iOS development efforts until their active users actually express their interest in joining the bandwagon.
</div>